In [61]:
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions


embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large-instruct"
)

client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection(
    name="documents_collection",
    embedding_function=embedding_model
)



In [62]:
import os
import json
import numpy as np


files = os.listdir('/home/wagyu0923/project/Document_Analyzer/embedded_files')

ids = []
meta_datas = []
embeddings = []
chunks = []

for file in files:
    with open(f'/home/wagyu0923/project/Document_Analyzer/embedded_files/{file}', 'r') as f:
        embedded_text = json.load(f)
    
    
    for idx, data in enumerate(embedded_text):
        meta_data = {}
        meta_data['source'] = file
        meta_data['page'] = data['page']
        meta_datas.append(meta_data)

        chunks.append(data['chunk'])

        embeddings.append(data['embedding'])

        ids.append(f'{file}_chunk_{idx}')


collection.add(
    embeddings = embeddings,
    metadatas = meta_datas,
    ids = ids,
    documents=chunks
)


In [125]:
query = input()

collected_data = collection.query(
    query_texts= [query],
    n_results=7,
)
retrived_data = ''

for meta, chunk in zip(collected_data['metadatas'][0], collected_data['documents'][0]):
    source = meta['source']
    page = meta['page']
    retrived_data += f'source : {source}, page : {page}, \n content : {chunk} \n\n'

In [126]:
import ollama

OLLAMA_MODEL = 'gpt-oss:20b'
SYSTEM_PROMPT = f"""You are an expert financial analyst. Your goal is to provide a clear and well-supported answer based *only* on the provided context.

**Instructions:**
1.  **Analyze and Synthesize:** Carefully analyze the user's QUESTION and the provided CONTEXT. Synthesize the information to infer a comprehensive answer.
2.  **Cite Sources:** For every piece of information or claim in your answer, you **must cite the specific `source` and `page`** it came from. For example: "The company was fined 2.7 billion won (Source: filename.json, Page: 214)."
3.  **Adhere to Context:** Base your answer **strictly** on the information within the CONTEXT. Do not use any outside knowledge.
4.  **Handle Missing Information:** If the CONTEXT does not contain enough information to answer the question, state that you cannot provide an answer based on the given documents.
5.  **Language:** Respond in Korean.

**CONTEXT:** {retrived_data}

**QUESTION:**
{query}
"""

response = ollama.chat(
    model=OLLAMA_MODEL,
    messages=[
        {'role': 'user', 'content': SYSTEM_PROMPT}
    ],
    options={

        'temperature': 0.1,
        'repeat_penalty': 1.1,
        'seed' : 42
    }
)
outputs = response['message']['content']

In [108]:
import ollama

OLLAMA_MODEL = 'gpt-oss:20b'
SYSTEM_PROMPT = f"""
컨텍스트 내용 정리해서 한국어로 답변 
CONTEXT: 
{retrived_data}
"""

response = ollama.chat(
    model=OLLAMA_MODEL,
    messages=[
        {'role': 'user', 'content': SYSTEM_PROMPT}
    ],
    options={

        'temperature': 0.1,
        'repeat_penalty': 1.1,
    }
)
outputs2 = response['message']['content']

In [127]:
print(outputs)

세토피아가 (주)제이슨앤컴퍼니를 합병한 주된 목적은 **철강전문기업과의 합병을 통해 신규 사업 진출과 매출 확대, 손익 구조 개선을 실현하여 기업가치와 주주가치를 제고**하는 데 있었습니다.  
(출처: 세토피아 반기보고서, 2025.09.09, 페이지 16)  

이 합병으로 인해 세토피아는 기존 사업에 **철강 전문 사업**을 추가하게 되어, 주된 사업 포트폴리오가 확장되었습니다. 즉, 세토피아는 (주)제이슨앤컴퍼니가 보유하고 있던 철강 전문 역량과 제품·서비스를 인수함으로써, 기존 사업 영역에 더해 철강 산업에 대한 진출과 매출 기반을 강화하게 됩니다.  
(출처: 세토피아 반기보고서, 2025.09.09, 페이지 16)


In [128]:
print(retrived_data)

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 20, 
 content : 합병으 로 인해 지급한 별도의 합병 교부금도 없습니다 본 합병 완료 후 (주)세토피아의 최대주주 변경은 없습니다 Ⅶ [합병등] 전ㆍ후의 요약재무정보 (단위 : 원) 구분 합병 전 합병 후 (주)세토피아 [합병회사] (주)제이슨앤컴퍼 니 [피합병회사] (주)세토피아 [합병 후 존속회사 ] 자산 Ⅰ 유동자산 17,256,827,953 21,897,369,048 32,338,897,275 

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 20, 
 content : 니다 Ⅵ 신주배정 등에 관한 사항 합병법인 (주)세토피아는 피합병법인 (주)제이슨앤컴퍼니의 지분을 100% 보유하고 있으며, 본 합병 완료 시 (주)세토피아는 존속회사로 계속 남아 있게 되며, 피합병법인 (주)제이슨앤컴퍼니는 해산하 여 소멸하게 됩니다 합병법인 (주)세토피아는 본 합병을 함에있어 신주를 발행하지 않으며, 합병으 로 인해 지급한 별도의 합병 교부금도 없습니다 본 합병 완료 후 (주)세토피아의 최대주주 변경은 없습니다 Ⅶ [합병등] 

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 16, 
 content : 1 합병방법 ㈜세토피아가 ㈜제이슨앤컴퍼니를 흡수합병 1) 존속회사: ㈜세토피아 2) 소멸회사: ㈜제이슨앤컴퍼니 - 합병형태 소규모합병 2 합병목적 철강전문기업과의 합병을 통한 신규사업 진출 및 매출확대, 손익구조 개선을 통한 기업가치 제고 및 주주가치 제고 3 합병의 중요영향 및 효과 (1) 회사의 경영에 미치는 영향 본 보고서 제출일 현재 합병법인인 (주)세토피아 는 피합병법인인 (주)제이슨앤컴퍼니의 발행주식 을 100% 소유하고 있으며, 합 

source : [세토피아][정정]반기보고서(2025.